In [3]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [1]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_201905_rtbfsearchengines_afterpublicconsultation_en.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/forgotten.md"

In [4]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 3, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [6]:
import re
import pandas as pd
file_path = "../original/forgotten.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [34]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX -- (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4])
df.reset_index(drop=True, inplace=True)

df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [35]:
import pandas as pd
from IPython.display import display

# Increase the display width
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

# Display the DataFrame slice
display(df[0:20])

,section,subsection,point,heading,text,section_reference
0,,,,False,INTRODUCTION,INTRODUCTION
1,,,,False,1. Following the Costeja judgment of the Court...,INTRODUCTION
2,,,,False,2. According to Google's Transparency Report[^...,INTRODUCTION
3,,,,False,"3. The European Data Protection Board (the ""ED...",INTRODUCTION
4,,,,False,"4. Accordingly, this document aims to interpre...",INTRODUCTION
5,,,,False,"5. Nonetheless, as under the Directive 95/46/E...",INTRODUCTION
6,,,,False,"6. As a preliminary point, it should be noted ...",INTRODUCTION
7,,,,False,7. There are some considerations when applying...,INTRODUCTION
8,,,,False,8. If a data subject obtains the delisting of ...,INTRODUCTION
9,,,,False,9. Delisting requests do not result in the per...,INTRODUCTION


In [36]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 1 augmented with footnotes
Row 2 augmented with footnotes
Row 5 augmented with footnotes
Row 7 augmented with footnotes
Row 12 augmented with footnotes
Row 36 augmented with footnotes
Row 41 augmented with footnotes
Row 63 augmented with footnotes
Row 77 augmented with footnotes
Row 78 augmented with footnotes
Row 79 augmented with footnotes
Row 82 augmented with footnotes
Row 83 augmented with footnotes
Row 100 augmented with footnotes


In [37]:
file = "../inputs/documents/forgotten.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [38]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.forgotten
importlib.reload(gdpr_rag.documents.forgotten)
from gdpr_rag.documents.forgotten import Forgotten

path_to_manual_as_csv_file = "../inputs/documents/Forgotten.parquet"
doc = Forgotten(path_to_manual_as_csv_file)


In [39]:
from IPython.display import Markdown, display

section = "1"
#section = "2.2.1"
section = "INTRODUCTION"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


INTRODUCTION

1. Following the Costeja judgment of the Court of Justice of the European Union ("CJEU") of the 13th of May 2014[^2], a data subject may request the provider of an online search engine ("search engine provider")[^3], to erase one or more links to web pages from the list of results displayed following a search made on the basis of his or her name.

2. According to Google's Transparency Report[^4], the percentage of URLs that Google has not delisted has not increased over the past 5 years since that judgement.  However, further to the CJEU judgement, data subjects seem to be more aware of their right to lodge a complaint for refusals of their delisting requests since Supervisory Authorities have observed an increase in the number of complaints regarding the refusal by search engine providers to delist links.

3. The European Data Protection Board (the "EDPB"), in accordance with its Action Plan, is developing guidelines in respect of Article 17 of the General Data Protection Regulation ("GDPR"). Until those guidelines are finalised, Supervisory Authorities must continue to handle and investigate, to the extent possible, complaints from data subjects and in a timely manner as possible.

4. Accordingly, this document aims to interpret the Right to be Forgotten in the search engines cases in light of the provisions of Article 17 GDPR (the "Right to request delisting"). Indeed, the Right to be Forgotten has been especially enacted under Article 17 GDPR to take into account the Right to request delisting established in the Costeja judgement.

5. Nonetheless, as under the Directive 95/46/EC of 24 October 1995 (the "Directive") and as stated by the CJEU in its aforementioned Costeja judgement[^5], the Right to request delisting implies two rights (Right to Object and Right to Erasure GDPR). Indeed, the application of Article 21 is expressly foreseen as the third ground for the Right to erasure. As a result, both Article 17 and Article 21 GDPR can serve as a legal basis for delisting requests. The right to object and the right to obtain erasure were already granted under the Directive. Nonetheless, as it will be addressed, the wording of the GDPR requires an adjustment of the interpretation of these rights.

6. As a preliminary point, it should be noted that, while Article 17 GDPR is applicable to all data controllers, this paper focuses solely on processing by search engine providers and delisting requests submitted by data subjects.

7. There are some considerations when applying Article 17 GDPR in respect of a search engine provider's data processing. In this regard, it is necessary to state that the processing of personal data carried out in the context of the activity of the search engine provider must be distinguished from processing that is carried out by the publishers of the third-party websites such as media outlets that provide online newspaper content[^6].

8. If a data subject obtains the delisting of a particular content, this will result in the deletion of that specific content from the list of search results concerning the data subject when the search is, as a main rule, based on his or her name. This content will however still be available using other search criteria.

9. Delisting requests do not result in the personal data being completely erased. Indeed, the personal data will neither be erased from the source website nor from the index and cache of the search engine provider. For example, a data subject may seek the delisting of personal data from a search engine's index which have originated from a media outlet, such as a newspaper article. In this instance, the link to the personal data may be delisted from the search engine's index; however, the article in question will still remain within the control of the media outlet and may remain publicly available and accessible, even if no longer visible in search results based on queries that include in principle the data subject's name.

10. Nevertheless, search engine providers are not exempt in a general manner from the duty to fully erase. In some exceptional cases, they will need to carry out actual and full erasure in their indexes or caches. For example, in the event that search engine providers would stop respecting robots.txt requests implemented by the original publisher, they would actually have a duty to fully erase the URL to the content, as opposed to delist which is mainly based on data subject's name.

11. This paper is divided into two topics. The first topic concerns the grounds a data subject can rely on for a delisting request sent to a search engine provider pursuant to Article 17.1 GDPR. The second topic concerns the exceptions to the Right to request delisting according to Article 17.3 GDPR. This paper will be supplemented by an appendix dedicated to the assessment of criteria for handling complaints for refusals of delisting.

12. This paper does not address Article 17.2[^7] GDPR. Indeed, this Article requires data controllers who have made the personal data public to inform controllers who have then reused those personal data through links, copies or replications. Such obligation of information does not apply to search engine providers when they find information containing personal data published or placed on the internet by third parties, index it automatically, store it temporarily and make it available to internet users according to a particular order of preference[^8]. In addition, it does not require search engine providers, who have received a data subject's delisting request, to inform the third party which made public that information on the internet. Such obligation seeks to give greater responsibility to original controllers and try to prevent from multiplying data subjects' initiatives. In this regard, the statement by the Article 29 Working Party, saying that search engine providers "should not as a general practice inform the webmasters of the pages affected by de-listing of the fact that some webpages cannot be acceded from the search engine in response to specific queries" because "such communication has no legal basis under EU data protection law"[^9] remains valid. It is also planned to have separate specific guidelines in respect of Article 17.2 GDPR.

  
[^2]: CJEU, Case C-131/12, Google Spain SL and Google Inc. v Agencia Española de Protección de Datos (AEPD) and Mario Costeja González, judgment of 13 May 2014.  
[^3]: including web archives such as archive.org  
[^4]: https://transparencyreport.google.com/eu-privacy/overview?hl=en  
[^5]: CJEU, Case C-131/12, judgment of 13 May 2014, paragraph 88: "Article 12(b) and subparagraph (a) of the first paragraph of Article 14 of Directive 95/46 are to be interpreted as meaning that, in order to comply with the rights laid down in those provisions and in so far as the conditions laid down by those provisions are in fact satisfied, the operator of a search engine is obliged to remove from the list of results displayed following a search made on the basis of a person's name links to web pages, published by third parties and containing information relating to that person, also in a case where that name or information is not erased beforehand or simultaneously from those web pages, and even, as the case may be, when its publication in itself on those pages is lawful".  
[^6]: CJEU, Case C 131/12, judgment of 13 May 2014; European Court of Human Rights (ECHR), "M.L. and W.W. vs Germany", 28 June 2018.  
[^7]: Regulation 2016/679 (GDPR), Article 17.2: "Where the controller has made the personal data public and is obliged pursuant to paragraph 1 to erase the personal data, the controller, taking account of available technology and the cost of implementation, shall take reasonable steps, including technical measures, to inform controllers which are processing the personal data that the data subject has requested the erasure by such controllers of any links to, or copy or replication of, those data."  
[^8]: See CJEU, Case C-136/17, GC and Others v CNIL, judgment of 24 September 2019, paragraph 35 and Case C-131/12, judgment of 13 May 2014, paragraph 41.  
[^9]: Article 29 Data Protection Working Party, "Guidelines on the implementation of the Court of Justice of the European Union judgment on "Google Spain and inc v. Agencia Española de Protección de Datos (AEPD) and Mario Costeja González" C- 131/12, WP 225, 26 November 2014, p. 23.